Image classification with CNNs
================

The goal of this exercise is to implement a specific CNN architecture with PyTorch and train it on the CIFAR-10 image classification dataset. We will start by introducing the dataset and then implement a `nn.Module` and a useful `Solver` class. Seperating the model from the actual training has proven itself as a sensible design decision. By the end of this exercise you should have succesfully trained your (possible) first CNN model and have a boilerplate `Solver` class which you can reuse for the next exercise and your future research projects.

For an inspiration on how to implement a model or the solver class you can have a look at [these](https://github.com/pytorch/examples) PyTorch examples.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
from torch.autograd import Variable
from data_utils import read_cancer_dataset
from data_utils import data_augmentation
from data_utils import norm_split_data

csv_full_name = '/home/ubuntu/dl4cvproject/data/train.csv'
img_folder_full_name = '/home/ubuntu/dl4cvproject/data/train_256'

# %matplotlib inline
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Cancer Dataset
=========

Since the focus of this exercise should be neural network models and how to successfully train them, we provide you with preprocessed and prepared datasets. For an even easier management of the train, validation and test data pipelines we provide you with custom `torch.utils.data.Dataset` classes. Use the official [documentation](http://pytorch.org/docs/data.html) to make yourself familiar with the `Dataset` and `DataLoader` classes. Think about how you have to integrate them in your training loop and have a look at the data preprocessing steps in `dl4cv/data_utils.py`.

The `num_workers` argument of the `DataLoader` class allows you to preprocess data with multiple threads.

<div class="alert alert-info">
    <h3>Note</h3>
    <p>In this case we generated the `Dataset` classes after we applied all the preprocessing steps. Other datasets or random data augmentation might require an online preprocessing which can be integrated into the `Dataset` classes. See `torchvision.Transform` for examples.</p>
</div>

In [6]:
data,fractions = read_cancer_dataset(csv_full_name=csv_full_name,img_folder_full_name=img_folder_full_name)

# print('store in local...')

# with open('raw_data.pickle', 'wb') as f_raw:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(raw_data, f_raw, pickle.HIGHEST_PROTOCOL)
    
# print('OK...')

  1%|          | 157/18577 [00:00<00:47, 386.07it/s]

bad image:  /home/ubuntu/dl4cvproject/data/train_256/scan_00010127.png total bad images:  1


  4%|▎         | 658/18577 [00:01<00:44, 405.60it/s]

bad image:  /home/ubuntu/dl4cvproject/data/train_256/scan_00010909.png total bad images:  2


  5%|▌         | 958/18577 [00:02<00:42, 411.36it/s]


Total good data size:  1000


  0%|          | 0/18577 [00:00<?, ?it/s]

OK...


In [10]:
aug_data = data_augmentation(data,fractions)

# print('Store augment data in local...')

# with open('aug_data.pickle', 'wb') as f_aug:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(aug_data, f_aug, pickle.HIGHEST_PROTOCOL)
    
# print('OK...')

0
OK
1
OK
2
OK
3
OK
4
OK
5
augment image i =  5
(256, 256, 3)


ValueError: all the input arrays must have same number of dimensions

In [ ]:
X_train,y_train,X_val,y_val,X_test,y_test = norm_split_data(aug_data)

print('Store good, augment, norm, split data in local...')

with open('data.pickle', 'wb') as f_final:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data, f_final, pickle.HIGHEST_PROTOCOL)
    
print('OK...')


In [ ]:
# with open('aug_data.pickle', 'rb') as f:
#     # The protocol version used is detected automatically, so we do not
#     # have to specify it.
#     loaddata = pickle.load(f)

Visualize Examples
------------------

To make yourself familiar with the dataset we visualize some examples. We show a few examples from each class. Note that we have to revert (transposition and mean subtraction) some preprocessing steps.

In [ ]:

index = 1899
img, _ = train_data[index]
print(img)
#print(img)
to_plot = img.cpu().numpy() #+ mean_image
print(to_plot.shape)
plt.imshow(to_plot[0])
plt.show()

## Model Architecture and Forward Pass 

After you understood the core concepts of PyTorch and have a rough idea on how to implement your own model, complete the initialization and forward methods of the `ClassificationCNN` in the `dl4cv/classifiers/classification_cnn.py` file. Note that we do not have to implement a backward pass since this is automatically handled by the `autograd` package.

Use the cell below to check your results:

## Training and Validation with the Solver
We train and validate our previously generated model with a seperate `Solver` class defined in `dl4cv/solver.py`. Complete the `.train()` method and try to come up with an efficient iteration scheme as well as an informative training logger.

Use the cells below to test your solver. A nice trick is to train your model with just a few training samples. You should be able to overfit small datasets, which will result in very high training accuracy and comparatively low validation accuracy.

<div class="alert alert-info">
    <h3>Note</h3>
    <p>As seen below, the design of our `Solver` class is indepdenent of the particular model or data pipeline. This facilitates the reuse of the class and its modular structure allows the training of different models.</p>
</div>

Plotting the loss, training accuracy, and validation accuracy should show clear overfitting:

## Train the Network
Now train your model with the full dataset. By training a `ThreeLayerCNN` model for one epoch, you should already achieve greater than 40% accuracy on the validation set. If your training is painfully slow check if you did not forget to call the `nn.Module.cuda()` method.

For the overfitting example we provided you with a set of hyperparamters (`hidden_dim`, `lr`, `weight_decay`, ...). You can start with the same parameter values but in order to maximize your accuracy you should try to train multiple models with different sets of hyperparamters. This process is called hyperparameter optimization.

In [ ]:
from yz.classifiers.classification_cnn import ClassificationCNN
from yz.classifiers.transferred_alexnet import alexnet
from yz.solver import Solver
from yz.data_utils import get_balanced_weights
from torchvision import models
import torch.nn as nn

weights = get_balanced_weights(train_label_list, 14)
#sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=40, shuffle=False, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=40, shuffle=False, num_workers=8)

model = alexnet(pretrained=True)
#model.classifier  = nn.Sequential(
#            nn.Linear(12544, 4096),
#            nn.ReLU(inplace=True),
#            nn.Linear(4096, 14),
#)
#model.classifier = nn.Sequential(
#            nn.Linear(12544, 4096),
#            nn.ReLU(inplace=True),
#            nn.Linear(4096, 4096),
#            nn.ReLU(inplace=True),
#            nn.Linear(4096, 14)
#)

#list(model.classifier.children())[:-1] = nn.Linear(4096, 14)  
#for param in model.features.parameters():
#    param.requires_grad = False
solver = Solver()
solver.train(model, train_loader, val_loader, log_nth=1, num_epochs=2)

# Test your Model
Run your best model on the test set. You should easily achieve a score above 10% (random guessing for a classification task with 10 classes) accuracy on the given test set:

In [ ]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=50, shuffle=False, num_workers=4)

scores = []
for inputs, target in tqdm(test_loader):
    #print(type(target))
    inputs, targets = Variable(inputs), Variable(target)
    #if model.is_cuda:
    #    inputs, targets = inputs.cuda(), targets.cuda()

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    scores.extend((preds == targets).data.cpu().numpy())
    
print('Test set accuracy: %f' % np.mean(scores))

## Get final test data

In [ ]:
from yz.data_utils import get_Cancer_datasets
csv_full_name = '/home/hpc/pr92no/ga42cih2/Projects/dl4cvproject/data/test.csv'
img_folder_full_name = '/home/hpc/pr92no/ga42cih2/Projects/dl4cvproject/data/test_400'
test_X, csv_test = get_Cancer_datasets(csv_full_name=csv_full_name,img_folder_full_name=img_folder_full_name, mode='upload')

In [ ]:
v = csv_test['detected'].values

In [ ]:
print(type(csv_test))
print(test_X.size())

In [ ]:
try:
    del csv_test['age']
except KeyError as e:
    print(e)
try:
    del csv_test['gender']
except KeyError as e:
    print(e)
try:
    del csv_test['view_position']
except KeyError as e:
    print(e)
try:
    del csv_test['image_name']
except KeyError as e:
    print(e)
try:
    del csv_test['detected']
except KeryError as e:
    print(e)

print(list(csv_test))

In [ ]:
inputs = test_X[1000:1020]
#if model.is_cuda:
#        inputs = inputs.cuda()
outputs = model(inputs)
_, preds = torch.max(outputs, 1)

## Prediction and Submission CSV

In [ ]:
import pandas as pd
index = 0
jump = 30
detected = []
pred_set = set()
for i in tqdm(range(int(test_X.size()[0] / jump) + 1)):
    start = index
    end = index + jump
    if end >= (test_X.size()[0]) :
        end = test_X.size()[0]
    inputs = test_X[start:end]
    # if model.is_cuda:
    #     inputs = inputs.cuda()
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    ###
    int_list_preds = preds.data.cpu().numpy().tolist()
    for pred_num in int_list_preds:
        pred_set.add(pred_num + 1)
    str_list_preds = [('class_' + str(pred_num + 1)) for pred_num in int_list_preds]
    detected.extend(str_list_preds)
    ####
    if end == test_X.size()[0]:
        break
    index += jump

In [ ]:
print(pred_set)
csv_test['detected'] = pd.Series(detected)
csv_test.to_csv('submission.csv', index=False)
print(csv_test)

model.save("models/classification_cnn.model")